# Analyse préliminaire et mise en contexte des données de Toflit18 pour le datasprint

Nous vous proposons un ensemble de visualisations permettant de remettre les données toflit18 sur la direction des Fermes de La Rochelle en 1789 dans le contexte de leur évolution sur le XVIIIème siècle, et de les comparer avec les autres directions des Fermes.

Nous vous proposons également un premier ensemble de visualisation permettant d'évaluer la part des exports et imports par bureaux de ferme, les échanges par partenaires et les produits qui y sont associés. 

Note : cette analyse préliminaire ne prend pas en compte les quantités/valeurs des flux, elle ne fait que compter en nombre de flux renseignés.

## Analyse diachronique des flux commerciaux de la direction des fermes de la Rochelle

In [ ]:
from poitousprint import Toflit
from vega import VegaLite
import pandas as pd

toflit_client = Toflit()

In [ ]:
flows_diachro = toflit_client.get_flows(start_year=1700, end_year=1800, customs_region='La Rochelle')

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux documentés pour la direction des fermes de La Rochelle dans le temps",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "year",
            "type": "quantitative", 
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de flux"
        },
        "color": {
            "field": "export_import",
            "type": "nominal",
            "title": "Type de flux"
        }
    }
}, pd.DataFrame(flows_diachro))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux documentés pour la direction des fermes de La Rochelle dans le temps, par source",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "year",
            "type": "quantitative", 
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de flux"
        },
        "color": {
            "field": "source_type",
            "type": "nominal",
            "title": "Type de source"
        }
    }
}, pd.DataFrame(flows_diachro))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux documentés pour la direction des fermes de La Rochelle, par partenaire commercial",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "year",
            "type": "quantitative", 
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de flux"
        },
        "color": {
            "field": "partner_grouping",
            "type": "nominal",
            "title": "Partenaire (classification 'grouping')"
        }
    }
}, pd.DataFrame(flows_diachro))

In [ ]:
# 5 types de produits les plus échangés sur le siècle

def build_top(year, products):
    products = [{"count": count, "product": product, "year": year} for product, count in products.items()]
    products = sorted(products, key=lambda d : d["count"], reverse=True)
    for index, p in enumerate(products):
        p["rank"] = index + 1
    return products[0:5]

def get_top_5(flows):
    flows_by_year = {}
    for f in flows:
        year = f['year'].split('.')[0]
        product = f['product_revolutionempire']
        if year not in flows_by_year:
            flows_by_year[year] = {}
        if product not in flows_by_year[year]:
            flows_by_year[year][product] = 1
        else:
            flows_by_year[year][product] = flows_by_year[year][product] + 1

    available_years = [int(y) for y in flows_by_year.keys()]
    available_years.sort()
    current_year = available_years[0]
    missing_years = []
    while current_year < available_years[-1]:
        if current_year not in available_years:
            missing_years.append({"count": 1, "product": "pas de données", "year": str(current_year), "rank": "n/a"})
        current_year += 1
    top_by_year = [build_top(year, products)  for year, products in flows_by_year.items()]
    top_by_year = [item for sublist in top_by_year for item in sublist]
    top_by_year += missing_years
    return top_by_year

VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution des 5 types de produits (révolution et empire) les plus échangés chaque année par la direction de La Rochelle",
    "mark": "rect",
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal",
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "title": "type de produit",
            "stack": "normalize",
            "sort": "color"
        },
        "color": {
            "field": "rank",
            "type": "ordinal",
            "title": "Rang dans les 5 produits les plus échangés de chaque année",
            "scale": {"scheme": "goldgreen", "reverse": True}
        }
    }
}, pd.DataFrame(get_top_5(flows_diachro)))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution des 5 types de produits (révolution et empire) les plus importés chaque année par la direction de La Rochelle",
    "mark": "rect",
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal",
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "title": "type de produit",
            "stack": "normalize",
            "sort": "color"
        },
        "color": {
            "field": "rank",
            "type": "ordinal",
            "title": "Rang dans les 5 produits les plus importés de chaque année",
            "scale": {"scheme": "goldgreen", "reverse": True}
        }
    }
}, pd.DataFrame(get_top_5([f for f in flows_diachro if f['export_import'] == 'Imports'])))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution des 5 types de produits (révolution et empire) les plus exportés chaque année par la direction de La Rochelle",
    "mark": "rect",
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal",
            "title": "année",
            "axis": {
                "format": "c"
            }
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "title": "type de produit",
            "stack": "normalize",
            "sort": "color"
        },
        "color": {
            "field": "rank",
            "type": "ordinal",
            "title": "Rang dans les 5 produits les plus exportés de chaque année",
            "scale": {"scheme": "goldgreen", "reverse": True}
        }
    }
}, pd.DataFrame(get_top_5([f for f in flows_diachro if f['export_import'] == 'Exports'])))

In [ ]:
"""
flows_diachro_all = toflit_client.get_flows(start_year=1700, end_year=1800, 
                                            params=[
                                                'customs_region', 
                                                'partner_simplification', 
                                                'partner_grouping',
                                                'product_revolutionempire',
                                                'year'
                                            ])

def parse_direction(f):
    direction = f['customs_region']
    if direction == '':
        f['direction'] = 'National'
    elif direction == 'La Rochelle':
        f['direction'] = 'La Rochelle'
    else:
        f['direction'] = 'Autre direction'
    return f
flows_diachro_all = [parse_direction(f) for f in flows_diachro_all]

"""
print('')

## Les flux de la direction des fermes de la Rochelle par rapport aux autres directions en 1789

In [ ]:
flows_1789= toflit_client.get_flows(year=1789)

In [ ]:
def resolve_direction(flow):
    f = flow.copy()
    if f['customs_region'] == '' or f['customs_region'] == 'National':
        f['customs_region'] = 'National'
    if f['export_import'] == 'import':
        f['export_import'] = 'Imports'
    return f
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par direction en 1789",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "y": {
            "field": "customs_region",
            "type": "nominal", 
            "title": "Bureau des fermes",
            "sort": "-x"
        },
        "x": {
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de flux"
        },
        "color": {
            "field": "export_import",
            "type": "nominal",
            "title": "Type de flux"
        }
    }
}, pd.DataFrame([resolve_direction(f) for f in flows_1789]))

In [ ]:
def resolve_direction(flow):
    f = flow.copy()
    if f['customs_region'] == '' or f['customs_region'] == 'National':
        f['customs_region'] = 'National'
    elif f['customs_region'] != 'La Rochelle':
        f['customs_region'] = 'Autres directions'
    return f
def compute_products_percentage_by_direction(flows):
    by_direction = [resolve_direction(f) for f in flows]

    m = {
        'National': {},
        'La Rochelle': {},
        'Autres directions': {}
    }
    for f in by_direction:
        product = f['product_revolutionempire']
        direction = f['customs_region']
        if product in m[direction]:
            m[direction][product] += 1
        else:
            m[direction][product] = 1
    products_ratio = [{"direction": key, "products": val} for key,val in m.items()]

    totals = {
        'National': len([p for p in by_direction if p['customs_region'] == 'National']),
        'La Rochelle': len([p for p in by_direction if p['customs_region'] == 'La Rochelle']),
        'Autres directions': len([p for p in by_direction if p['customs_region'] == 'Autres directions'])
    }

    products_by_direction = []
    for products in products_ratio:
        direction = products["direction"]
        these_products = products["products"]
        products_by_direction = products_by_direction + [{"product": product, "direction": direction, "percentage": count / totals[direction] * 100} for product, count in these_products.items()]

    return products_by_direction

products_by_directions_all = compute_products_percentage_by_direction([resolve_direction(f) for f in flows_1789])
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits par direction pour 1789",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque type de produit dans les flux"
        }
    }
}, pd.DataFrame(products_by_directions_all))

In [ ]:
products_by_directions_imports = compute_products_percentage_by_direction([resolve_direction(f) for f in flows_1789 if f['export_import'] == 'import' or f['export_import'] == 'Imports'])

VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits par direction pour 1789 (imports seulement)",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque type de produit dans les flux"
        }
    }
}, pd.DataFrame(products_by_directions_imports))

In [ ]:
products_by_directions_exports = compute_products_percentage_by_direction([f for f in flows_1789 if f['export_import'] == 'Exports'])
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits par direction pour 1789 (exports seulement)",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque type de produit dans les flux"
        }
    }
}, pd.DataFrame(products_by_directions_exports))

In [ ]:
def resolve_direction(flow):
    f = flow.copy()
    if f['customs_region'] == '' or f['customs_region'] == 'National':
        f['customs_region'] = 'National'
    elif f['customs_region'] != 'La Rochelle':
        f['customs_region'] = 'Autres directions'
    return f
def compute_partners_percentage_by_direction(flows):
    by_direction = [resolve_direction(f) for f in flows]

    m = {
        'National': {},
        'La Rochelle': {},
        'Autres directions': {}
    }
    for f in by_direction:
        partner = f['partner_simplification']
        direction = f['customs_region']
        if partner in m[direction]:
            m[direction][partner] += 1
        else:
            m[direction][partner] = 1
    partners_ratio = [{"direction": key, "partners": val} for key,val in m.items()]

    totals = {
        'National': len([p for p in by_direction if p['customs_region'] == 'National']),
        'La Rochelle': len([p for p in by_direction if p['customs_region'] == 'La Rochelle']),
        'Autres directions': len([p for p in by_direction if p['customs_region'] == 'Autres directions'])
    }

    partners_by_direction = []
    for partners in partners_ratio:
        direction = partners["direction"]
        these_partners = partners["partners"]
        partners_by_direction = partners_by_direction + [{"partner": partner, "direction": direction, "percentage": count / totals[direction] * 100} for partner, count in these_partners.items()]

    return partners_by_direction
flows_with_binned_direction = [resolve_direction(f) for f in flows_1789]
partners_by_direction_all = compute_partners_percentage_by_direction([f for f in flows_with_binned_direction])
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des partenaires par direction pour 1789",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "partner",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque partenaire commercial dans les flux"
        }
    }
}, pd.DataFrame(partners_by_direction_all))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des partenaires par direction pour 1789 (imports seulement)",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "partner",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque partenaire commercial dans les flux"
        }
    }
}, pd.DataFrame(compute_partners_percentage_by_direction([resolve_direction(f) for f in flows_1789 if f['export_import'] == 'import' or f['export_import'] == 'Imports'])))


In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des partenaires par direction pour 1789 (export seulement)",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "direction",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "partner",
            "sort": "-size"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Origine du flux"
        },
        "size": {
            "type": "quantitative",
            "field": "percentage",
            "title": "Pourcentage de chaque partenaire commercial dans les flux"
        }
    }
}, pd.DataFrame(compute_partners_percentage_by_direction([resolve_direction(f) for f in flows_1789 if f['export_import'] == 'Exports'])))

## Premier aperçu des données de la direction de La Rochelle en 1789

In [ ]:
la_rochelle = toflit_client.get_flows(customs_region='La Rochelle', year=1789)
for f in la_rochelle:
    if f['customs_office'] == '':
        f['customs_office'] = 'Pas de bureau spécifié'

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par bureau des fermes",
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative",
            "aggregate": "count"
        },
        "y": {
            "type": "nominal",
            "field": "customs_office",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "export_import",
            "title": "Type de flux"
        }
    }
}, pd.DataFrame(la_rochelle))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits exports par bureau (classification révolution et empire)",
    "mark": "circle",
    "encoding": {
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "title": "production (classification 'révolution et empire')",
            "sort": "-size"
        },
        "x": {
            "type": "nominal",
            "field": "customs_office",
            "title": "bureau des fermes",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre d'exports",
            "aggregate": "count"
        },
        "color": {
            "type": "nominal",
            "field": "customs_office",
            "title": "bureau des fermes"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Exports']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits importés par bureau (classification révolution et empire)",
    "mark": "circle",
    "encoding": {
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "title": "production (classification 'révolution et empire')",
            "sort": "-size"
        },
        "x": {
            "type": "nominal",
            "field": "customs_office",
            "title": "bureau des fermes",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre d'imports",
            "aggregate": "count"
        },
        "color": {
            "type": "nominal",
            "field": "customs_office",
            "title": "bureau des fermes"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Imports']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Partenaires commerciaux de la région (simplification)",
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative",
            "title": "nombre de flux",
            "aggregate": "count"
        },
        "y": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "partenaire (classification simplification)",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "export_import"
        }
    }
}, pd.DataFrame(la_rochelle))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Partenaires privilégiés pour les exports par bureau de ferme",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "Partenaire (classification simplification)",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "customs_office",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre de flux d'exports",
            "aggregate": "count"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Exports']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Partenaires privilégiés pour les imports par bureau de ferme",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "Partenaire (classification simplification)",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "customs_office",
            "title": "Bureau des fermes",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre de flux d'imports",
            "aggregate": "count"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Imports']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Types de produits exportés par partenaire",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "partenaire (classification 'simplification')",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "title": "type de produit (classification 'révolution et empire')",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre de flux d'exports",
            "aggregate": "count"
        },
        "color": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "partenaire (classification 'simplification')"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Exports']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Types de produits importés par partenaire",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "partenaire (classification 'simplification')",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "title": "type de produit (classification 'révolution et empire')",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "title": "nombre de flux d'import",
            "aggregate": "count"
        },
        "color": {
            "type": "nominal",
            "field": "partner_simplification",
            "title": "Partenaire de commerce"
        }
    }
}, pd.DataFrame([f for f in la_rochelle if f['export_import'] == 'Imports']))

## Visualisations en réseau des données de la direction des fermes de La Rochelle en 1789

In [ ]:
import json
import networkx as nx
from ipysigma import Sigma


# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

Remarques : certains des réseaux bipartites présentés ci-après peuvent également être obtenus grâce au [datascape Toflit18](http://toflit18.medialab.sciences-po.fr/#/home).

In [ ]:
"""
#### Définition de fonctions génériques qui permettent de fabriquer des réseaux bipartites et triparties à partir :

1. d'une liste de dicts (ex. flux toflit18)
2. d'une liste de propriétés (2 pour les réseaux bipartites, 3 pour les tripartites)
3. d'un param filter_import_export que l'on peut mettre sur import ou exports (par défaut on a à la fois les imports et les exports
"""
def render_coocurrences_graph(data, key_1, key_2, filter_export_import=None, params=None, dump_gexf_to=None):
    if filter_export_import not in [None, 'Exports', 'Imports']:
        print("filter_export_import must be 'Exports', 'Imports' or None")
        raise
    
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)",
        "color_2": "rgb(255, 0, 0)",
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
    
    # remplir les dicts
    for datum in data:
        if (filter_export_import is not None and datum['export_import'] != filter_export_import):
            continue
            
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            
            edge_footprint = value_1_id + "-" + value_2_id
            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
                
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)
    if dump_gexf_to is not None:
        nx.write_gexf(Graph, dump_gexf_to)

    return Sigma(Graph, start_layout=True)

In [ ]:
def render_tripartite_graph(data, key_1, key_2, key_3, filter_export_import=None, params=None, dump_gexf_to=None):
    if filter_export_import not in [None, 'Exports', 'Imports']:
        print("filter_export_import must be 'Exports', 'Imports' or None")
        raise
    
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {} # -> products
    key2_uniq = {} # -> partners
    key3_uniq = {} # -> customs_office
    edges_uniq = {}
    
    default_params = {
        "color_1": "rgb(0, 255, 0)", # noeuds de type 1 en vert
        "color_2": "rgb(255, 0, 0)", # noeuds de type 2 en rouge
        "color_3": "rgb(0, 0, 255)", # noeuds de type 3 en bleu
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
        
    # remplir les dicts
    for datum in data:
        
        if (filter_export_import is not None and datum['export_import'] != filter_export_import):
            continue
            
        if key_1 in datum and key_2 in datum and key_3 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_3 = datum[key_3] if datum[key_3] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            value_3_id = key_3 + "_" + value_3
            
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            if value_3_id in key3_uniq:
                key3_uniq[value_3_id] = {**key3_uniq[value_3_id], "size": key3_uniq[value_3_id]["size"] + 1}
            else:
               key3_uniq[value_3_id] = {
                   "type": key_3, 
                   "name": value_3, 
                   "color": final_params["color_3"],
                   "size": 1
               }
            
            edge_footprints = {
                str(value_1_id + "-" + value_2_id): {
                    'source': value_1_id,
                    'target': value_2_id
                }, 
                str(value_2_id + "-" + value_3_id): {
                    'source': value_2_id,
                    'target': value_3_id
                }, 
                str(value_1_id + "-" + value_3_id): {
                    'source': value_1_id,
                    'target': value_3_id
                }
            } 

            for edge_footprint in edge_footprints.keys():
                if edge_footprint in edges_uniq:
                    edges_uniq[edge_footprint]["weight"] += 1
                else:
                    edges_uniq[edge_footprint] = {
                        "source": edge_footprints[edge_footprint]['source'],
                        "target": edge_footprints[edge_footprint]['target'],
                        "weight": 1
                    }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    all_nodes.update(key3_uniq) 

    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [1, 10]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)
    
    if dump_gexf_to is not None:
        nx.write_gexf(Graph, dump_gexf_to)

    return Sigma(Graph, start_layout=True)

In [ ]:
"""
# 0. Je vais chercher les flux du sprint (côté Toflit) pour nourrir mes réseaux

nb : pour les flux concernant le sprint on n'a qu'une seule direction des Fermes : La Rochelle
"""
# je vais chercher les flux qui concernent le sprint côté Toflit

flows = toflit_client.get_flows(
    year=1789,
    customs_region='La Rochelle', 
    params=[
      "product_revolutionempire",
      "partner",
      "export_import",
      "value",
      "line",
      "partner_simplification",
      "customs_office",
      "customs_region"
	]
)

print("Nombre de flows concernant le sprint trouvés : ", len(flows))

### Réseau bipartite des bureaux des Fermes et les partenaires commerciaux 

### Imports uniquement



In [ ]:
"""
On rentre dans un niveau de détail plus élévé en associant les 7 bureaux des Fermes concernés par le sprint avec les partenaires commerciaux avec lesquels ils réalisent des imports.
"""
render_coocurrences_graph(flows, "customs_office", "partner_simplification", filter_export_import = 'Imports', dump_gexf_to='dumps/visualisations/toflit/toflit_bipartite_fermes_partners_imports.gexf')


<img src="dumps/visualisations/toflit/toflit_bipartite_2.png">

<h4>
    <a target="blank"
       rel="noopener"
       href="https://robindemourat.github.io/gexf-fast-preview/?data=https://raw.githubusercontent.com/medialab/portic-datasprint-2021/main/preliminary_inquiry/dumps/visualisations/toflit/toflit_bipartite_fermes_partners_imports.gexf"
     >Voir cette visualisation en version interactive plein écran</a>
</h4>

### Exports uniquement

In [ ]:
"""
# Flux Toflit concernés par le sprint - Réseau bipartite directions des Fermes / partenaires commerciaux avec qui on exporte

#### Même réseau que le précédent, sauf que les partenaires commerciaux représentés sont ceux vers qui les bureaux de Ferme exportent
"""
render_coocurrences_graph(flows, "customs_office", "partner_simplification", filter_export_import = 'Exports', dump_gexf_to='dumps/visualisations/toflit/toflit_bipartite_fermes_partners_exports.gexf')

<img src="dumps/visualisations/toflit/toflit_bipartite_3.png">

<h4>
    <a target="blank"
       rel="noopener"
       href="https://robindemourat.github.io/gexf-fast-preview/?data=https://raw.githubusercontent.com/medialab/portic-datasprint-2021/main/preliminary_inquiry/dumps/visualisations/toflit/toflit_bipartite_fermes_partners_exports.gexf"
     >Voir cette visualisation en version interactive plein écran</a>
</h4>

### Réseau tripartite entre les bureaux des Fermes, les produits et les partenaires commerciaux



In [ ]:
"""
Objectifs : 

- commencer à distinguer les produits exportés des produits importés (pour les flux concernant le sprint)
- identifier des clusters (par exemple partenaire / produits)
"""
print('')

### Imports uniquement

In [ ]:
"""
# Flux Toflit concernés par le sprint - Réseau tripartite produits / directions des Fermes / partenaires commerciaux avec qui on importe

#### Association des bureaux des Fermes avec les produits importés et les partenaires commerciaux avec lesquels ces imports se font 

"""

render_tripartite_graph(flows, "partner", "product_revolutionempire", "customs_office", filter_export_import = 'Imports', dump_gexf_to='dumps/visualisations/toflit/toflit_tripartite_fermes_partners_products_imports.gexf')



<img src="dumps/visualisations/toflit/toflit_tripartite_1.png">


<h4>
    <a target="blank"
       rel="noopener"
       href="https://robindemourat.github.io/gexf-fast-preview/?data=https://raw.githubusercontent.com/medialab/portic-datasprint-2021/main/preliminary_inquiry/dumps/visualisations/toflit/toflit_tripartite_fermes_partners_products_imports.gexf"
     >Voir cette visualisation en version interactive plein écran</a>
</h4>



#### Légende

Bleu : bureaux de ferme

Rouge : produits (classification révolution & empire)

Vert : partenaire commercial

### Exports uniquement

In [ ]:
"""
# Flux Toflit concernés par le sprint - Réseau bipartite directions des Fermes / partenaires commerciaux avec qui on exporte

#### Même réseau que le précédent, sauf que les produits sont exportés et les partenaires commerciaux représentés sont ceux avec qui les bureaux des Fermes exportent
"""
render_tripartite_graph(flows, "partner", "product_revolutionempire", "customs_office", filter_export_import = 'Exports', dump_gexf_to='dumps/visualisations/toflit/toflit_tripartite_fermes_partners_products_exports.gexf')



<img src="dumps/visualisations/toflit/toflit_tripartite_2.png">

<h4>
    <a target="blank"
       rel="noopener"
       href="https://robindemourat.github.io/gexf-fast-preview/?data=https://raw.githubusercontent.com/medialab/portic-datasprint-2021/main/preliminary_inquiry/dumps/visualisations/toflit/toflit_tripartite_fermes_partners_products_exports.gexf"
     >Voir cette visualisation en version interactive plein écran</a>
</h4>


#### Légende

Bleu : bureaux de ferme

Rouge : produits (classification révolution & empire)

Vert : partenaire commercial